This script takes as inputs: 
- file geojson of Milan areas
- output.csv with all the restaurants scraped (restaurant_id is set to be equal to the row number)
- nil.csv file with the population per area of Milan

It produces three datasets:
- output_cleaned, with restaurant_id, location, revenue (where missing, the mean value for the same category of restaurant was imputed), category, reviews, rating
- rest_locationinfo.csv,  dataset with restaurant_id, lat, long, ID_NIL (Milan area ID), NIL (Milan area)
- quartieri.csv, with ID_NIL, population for NIL, density or restaurants per NIL

**Cleaning dataset**

In [1]:
import time
import requests
import json
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from pandas import DataFrame

In [2]:
df = pd.read_csv("output.csv")
df['restaurant_id']= df.reset_index().index
df=df.set_index('restaurant_id')
print(df.head(10))

                                                             URL  ...                                            Cuisine
restaurant_id                                                     ...                                                   
0              https://www.tripadvisor.com/Restaurant_Review-...  ...                                  Filipino, Healthy
1              https://www.tripadvisor.com/Restaurant_Review-...  ...  Middle Eastern, Turkish, Pub, Bar, Internation...
2              https://www.tripadvisor.com/Restaurant_Review-...  ...                      Italian, Steakhouse, Barbecue
3              https://www.tripadvisor.com/Restaurant_Review-...  ...                    Italian, Street Food, Fast Food
4              https://www.tripadvisor.com/Restaurant_Review-...  ...  Italian, Mediterranean, Deli, Street Food, Apu...
5              https://www.tripadvisor.com/Restaurant_Review-...  ...  Italian, Pizza, Mediterranean, Neapolitan, Cam...
6              https://www.tripa

In [3]:
df = DataFrame(df, columns=['URL','Title','Location','No. of Reviews','Rating','Price Range','Cuisine'])
df = df.rename(columns={'No. of Reviews': 'Reviews', 'Price Range': 'Price'})
df.head()

,URL,Title,Location,Reviews,Rating,Price,Cuisine
restaurant_id,,,,,,,
0,https://www.tripadvisor.com/Restaurant_Review-...,Mabuhay Restaurant,"Bastioni di Porta Volta 9, 20121 Milan Italy",368,5.0,$2 - $14,"Filipino, Healthy"
1,https://www.tripadvisor.com/Restaurant_Review-...,Star Zagros Kebabbar,"Corso 22 Marzo, 38, 20135 Milan Italy",236,5.0,NaN,"Middle Eastern, Turkish, Pub, Bar, Internation..."
2,https://www.tripadvisor.com/Restaurant_Review-...,Shabby Grill Restaurant,"Via Domokos 4, 20147 Milan Italy",174,5.0,$48 - $59,"Italian, Steakhouse, Barbecue"
3,https://www.tripadvisor.com/Restaurant_Review-...,Il Panino del Laghetto,"Via Laghetto 7, 20122 Milan Italy",168,5.0,$5 - $14,"Italian, Street Food, Fast Food"
4,https://www.tripadvisor.com/Restaurant_Review-...,Panzarotti,"Viale Bligny 1/A Piazzale di Porta Lodovica, 2...",665,5.0,$5 - $18,"Italian, Mediterranean, Deli, Street Food, Apu..."


In [4]:
df['A'], df['B'] = df['Price'].str.split('-', 1).str
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 0 to 2442
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   URL       2443 non-null   object 
 1   Title     2443 non-null   object 
 2   Location  2443 non-null   object 
 3   Reviews   2443 non-null   object 
 4   Rating    2443 non-null   float64
 5   Price     1059 non-null   object 
 6   Cuisine   2392 non-null   object 
 7   A         1059 non-null   object 
 8   B         1059 non-null   object 
dtypes: float64(1), object(8)
memory usage: 190.9+ KB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


In [5]:
df['A'] = pd.to_numeric(df['A'].astype(str).str.replace('$', ''), errors='coerce')
df['B'] = pd.to_numeric(df['B'].astype(str).str.replace('$', ''), errors='coerce')
df['Price'] = (df['A']+df['B'])/2
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')
df['Revenue'] = df['Price'] * df['Reviews']
del df['A']
del df['B']

In [7]:
df['Category'] = df['Cuisine'].str.partition(',')[0]
df=df.dropna(subset = ['Category'])
df["Revenue"] = df.groupby("Category").transform(lambda x: x.fillna(x.mean()))
df["Price"] = df.groupby("Category").transform(lambda x: x.fillna(x.mean()))
df.to_csv('output_cleaned.csv') 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Adding Lat and Long to addresses in the Scraped Df

In [16]:
restaurant_location = []
for restaurant_id, restaurant in tqdm(df.iterrows(), total=df.shape[0]):
  address = restaurant['Location']
  if(address is None):
    print(str(restaurant_id) + "- " + restaurant['location'])
  else:
    key = "0btw1lSfowFGAKb5of9Os2XpzRsk8G7Y"
    try:
      geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={address}"
      response = requests.get(geocode_url)
      if(response.status_code == 200):
        geo = json.loads(response.text)
        lat = geo['results'][0]['locations'][0]['latLng']['lat']
        lng = geo['results'][0]['locations'][0]['latLng']['lng']
        restaurant_location.append({
              "restaurant_id": restaurant_id,
              "lat": lat,
              "lng": lng
            })
    except Exception as e:
      print(e)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2392 [00:00<?, ?it/s]

In [17]:
print(restaurant_location)

[{'restaurant_id': 0, 'lat': 45.4794, 'lng': 9.18209}, {'restaurant_id': 1, 'lat': 45.46201, 'lng': 9.21903}, {'restaurant_id': 2, 'lat': 45.46848, 'lng': 9.12568}, {'restaurant_id': 3, 'lat': 45.46162, 'lng': 9.19556}, {'restaurant_id': 4, 'lat': 45.45179, 'lng': 9.18687}, {'restaurant_id': 5, 'lat': 45.45298, 'lng': 9.17189}, {'restaurant_id': 6, 'lat': 45.45381, 'lng': 9.16346}, {'restaurant_id': 7, 'lat': 45.47149, 'lng': 9.18479}, {'restaurant_id': 8, 'lat': 45.48343, 'lng': 9.2154}, {'restaurant_id': 9, 'lat': 45.45984, 'lng': 9.19142}, {'restaurant_id': 10, 'lat': 45.4491, 'lng': 9.17987}, {'restaurant_id': 11, 'lat': 45.4265, 'lng': 9.25098}, {'restaurant_id': 12, 'lat': 45.45874, 'lng': 9.17814}, {'restaurant_id': 13, 'lat': 45.46753, 'lng': 9.15077}, {'restaurant_id': 14, 'lat': 45.4906, 'lng': 9.20296}, {'restaurant_id': 15, 'lat': 45.4777, 'lng': 9.19893}, {'restaurant_id': 16, 'lat': 45.47585, 'lng': 9.18945}, {'restaurant_id': 17, 'lat': 45.46245, 'lng': 9.18544}, {'resta

In [18]:
ds_restaurant_location = pd.DataFrame(restaurant_location)
ds_restaurant_location.set_index("restaurant_id")
ds_restaurant_location.head(10)

,restaurant_id,lat,lng
0,0,45.47940,9.18209
1,1,45.46201,9.21903
2,2,45.46848,9.12568
3,3,45.46162,9.19556
4,4,45.45179,9.18687
5,5,45.45298,9.17189
6,6,45.45381,9.16346
7,7,45.47149,9.18479
8,8,45.48343,9.21540
9,9,45.45984,9.19142


Adding 'Quartiere'

In [19]:
!pip install geopandas
!pip install rtree
!pip install pygeos
import pygeos
import rtree
import geopandas as gpd
!sudo apt install libspatialindex-dev
!pip install --upgrade geopandas

     |████████████████████████████████| 994 kB 6.5 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 6.3 MB 49.0 MB/s 
     |████████████████████████████████| 994 kB 7.7 MB/s 
     |████████████████████████████████| 1.8 MB 7.8 MB/s 


/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 kB]
Fetched 555 kB in 1s (429 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.

In [20]:
# 1-  Listing points
listings = ds_restaurant_location
# 2 - convert to Geopandas Geodataframe
gdf_listings = gpd.GeoDataFrame(listings,   geometry=gpd.points_from_xy(listings.lng, listings.lat))
# 3 - Neighbourhoods
milan_areas = gpd.read_file('ds964_nil_wm.geojson')
milan_areas.head()

,ID_NIL,NIL,Valido_dal,Valido_al,Fonte,Shape_Length,Shape_Area,OBJECTID,geometry
0,48,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,8723.368714,2.406306e+06,89,"POLYGON ((9.15422 45.43775, 9.15274 45.43887, ..."
1,64,TRENNO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,3309.998800,4.896921e+05,90,"POLYGON ((9.10623 45.49016, 9.10591 45.49084, ..."
2,67,PORTELLO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,3800.750663,9.096022e+05,91,"POLYGON ((9.15636 45.48785, 9.15495 45.48852, ..."
3,81,BOVISASCA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,7105.469715,1.578028e+06,92,"POLYGON ((9.16803 45.52234, 9.16763 45.52272, ..."
4,84,PARCO NORD,05/02/2020,Vigente,Milano 2030 - PGT Approvato,11741.717005,1.532331e+06,93,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."


In [21]:
sjoined_listings = gpd.sjoin(gdf_listings, milan_areas, op="within")
sjoined_listings.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  """Entry point for launching an IPython kernel.


,restaurant_id,lat,lng,geometry,index_right,ID_NIL,NIL,Valido_dal,Valido_al,Fonte,Shape_Length,Shape_Area,OBJECTID
0,0,45.47940,9.18209,POINT (9.18209 45.47940),84,2,BRERA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5461.667196,1.637333e+06,173
7,7,45.47149,9.18479,POINT (9.18479 45.47149),84,2,BRERA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5461.667196,1.637333e+06,173
16,16,45.47585,9.18945,POINT (9.18945 45.47585),84,2,BRERA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5461.667196,1.637333e+06,173
35,35,45.46948,9.18211,POINT (9.18211 45.46948),84,2,BRERA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5461.667196,1.637333e+06,173
50,50,45.47789,9.18899,POINT (9.18899 45.47789),84,2,BRERA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5461.667196,1.637333e+06,173


In [22]:
sjoined_listings.to_csv('rest_locationinfo.csv') 

In [23]:
grouped = sjoined_listings.groupby('ID_NIL').size()
df_nil = grouped.to_frame().reset_index()
df_nil.columns = ['ID_NIL', 'restaurants_count']
df_nil=df_nil.set_index('ID_NIL')
temp = sjoined_listings[['ID_NIL','NIL']]
temp=temp.set_index('ID_NIL')
result = temp.join(df_nil)
result = result.reset_index()
result = result.drop_duplicates(subset=['ID_NIL'], keep='first')
print(result)

      ID_NIL                                  NIL  restaurants_count
0          1                                DUOMO                205
205        2                                BRERA                114
319        4                            GUASTALLA                 55
374        5     PORTA VIGENTINA - PORTA LODOVICA                 44
418        6  PORTA TICINESE - CONCA DEL NAVIGLIO                 81
...      ...                                  ...                ...
2258      82                             COMASINA                  1
2259      83                             BRUZZANO                  2
2261      85                  PARCO DELLE ABBAZIE                  1
2262      87                              ASSIANO                  1
2263      88                PARCO BOSCO IN CITTA'                  2

[79 rows x 3 columns]


In [24]:
df_cleaned=df
df = pd.read_csv("nil.csv")
temp = sjoined_listings[['ID_NIL','NIL']]
temp=temp.set_index('NIL')

In [25]:
df=df.set_index('NIL')
result1 = temp.join(df)
result1 = result1.reset_index()
result1 = result1.drop_duplicates(subset=['NIL'], keep='first')

In [26]:
result = result.set_index('ID_NIL')
result1 = result1.set_index('ID_NIL')

In [27]:
result2=pd.concat([result, result1], axis=1)
result2=result2.reset_index()
result2.to_csv('quartieri.csv') 

In [33]:
sjoined_listings.head()
restloc = DataFrame(sjoined_listings,columns=['restaurant_id','lat','lng','ID_NIL','NIL'])

In [34]:
restloc.head()

,restaurant_id,lat,lng,ID_NIL,NIL
0,0,45.47940,9.18209,2,BRERA
7,7,45.47149,9.18479,2,BRERA
16,16,45.47585,9.18945,2,BRERA
35,35,45.46948,9.18211,2,BRERA
50,50,45.47789,9.18899,2,BRERA


In [41]:
result3 = result2.loc[:,~result2.columns.duplicated()]
result3=result3.drop_duplicates(subset=['ID_NIL'], keep='first')

In [43]:
result3 = DataFrame(result3,columns=['ID_NIL','restaurants_count','Population','Pricexm2'])

In [69]:
result4 = pd.merge(restloc, result3, on='ID_NIL')

In [72]:
df_cleaned = DataFrame(df_cleaned,columns=['Title','Reviews','Rating','Price','Revenue','Category'])

In [73]:
df_cleaned = df_cleaned.reset_index()

In [74]:
result5 = pd.merge(df_cleaned, result4, on='restaurant_id')

In [75]:
result5.head()

,restaurant_id,Title,Reviews,Rating,Price,Revenue,Category,ID_NIL,lat,lng,NIL,restaurants_count,Population,Pricexm2,Pricexm2a
0,0,Mabuhay Restaurant,368.0,5.0,8.0,368.0,Filipino,2,45.47940,9.18209,BRERA,114,18492,6250,38.0
1,1,Star Zagros Kebabbar,236.0,5.0,NaN,236.0,Middle Eastern,26,45.46201,9.21903,XXII MARZO,89,31720,4100,26.0
2,2,Shabby Grill Restaurant,174.0,5.0,53.5,174.0,Italian,56,45.46848,9.12568,FORZE ARMATE,8,24871,1700,NaN
3,3,Il Panino del Laghetto,168.0,5.0,9.5,168.0,Italian,1,45.46162,9.19556,DUOMO,205,16765,4950,36.0
4,4,Panzarotti,665.0,5.0,11.5,665.0,Italian,5,45.45179,9.18687,PORTA VIGENTINA - PORTA LODOVICA,44,13686,3300,24.0


In [79]:
result5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2265 entries, 0 to 2264
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      2265 non-null   int64  
 1   Title              2265 non-null   object 
 2   Reviews            2169 non-null   float64
 3   Rating             2265 non-null   float64
 4   Price              2265 non-null   float64
 5   Revenue            2265 non-null   float64
 6   Category           2265 non-null   object 
 7   ID_NIL             2265 non-null   int64  
 8   lat                2265 non-null   float64
 9   lng                2265 non-null   float64
 10  NIL                2265 non-null   object 
 11  restaurants_count  2265 non-null   int64  
 12  Population         2265 non-null   int64  
 13  Pricexm2           2265 non-null   int64  
 14  Pricexm2a          1728 non-null   float64
dtypes: float64(7), int64(5), object(3)
memory usage: 283.1+ KB
